In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#plt.style.use('seaborn')
#plt.rcParams['axes.unicode_minus'] = False
#plt.rcParams['font.family'] = 'SimHei'
#%matplotlib inline 

#plt.rcParams['xtick.direction'] = 'inout'  # x軸刻度 位置
#plt.rcParams['xtick.major.size'] = 3.5 # x軸刻度 長度

In [2]:
#臺中市政府警察局106年11月份交通事故資料
url_10611 = 'https://quality.data.gov.tw/dq_download_csv.php?nid=72293&md5_url=27099e7b514bec2a041fd8d0194fbd69'

#臺中市政府警察局106年10月份交通事故資料
url_10610 = 'https://quality.data.gov.tw/dq_download_csv.php?nid=59792&md5_url=351eb4516b664646e487f4b7ff5d6671'

In [3]:
# 使用pandas讀取網路連結檔案
df_10611 = pd.read_csv(url_10611)
df_10610 = pd.read_csv(url_10610)

# 使用pandas 存檔
#df_10611.to_csv('交通事故10611.csv')
#df_10610.to_csv('交通事故10610.csv')

## 連接資料庫

In [112]:
# 資料庫連接
# CHARSET=UTF8; 和 cn.setencoding(encoding='utf-8') 都要設定

import pyodbc
cn = pyodbc.connect("DRIVER={MySQL ODBC 8.0 ANSI Driver}; SERVER=localhost; PORT=3306;DATABASE=taichung; UID=root; PASSWORD=123456;OPTION=3;CHARSET=UTF8;") 
cn.setencoding(encoding='utf-8')
cursor = cn.cursor()

In [541]:
#cursor.execute("create database taichung;")
cursor.execute("use taichung;")
cursor.execute("show tables;")
print(cursor.fetchall())
#cursor.commit()
#cn.close()

[('pls', ), ('qtable', ), ('taichung_', ), ('taichung_201711', ), ('tc', ), ('tc_lite', )]


### 資料內容    　　　  chat: 縣市、區、速限(非道路)  　　　其他都是數字

###  ===較少的欄位 寫入資料庫OK===

In [127]:
df = df_10611[['年', '月','日', '時', '分', '縣市', '區', 'GPS座標X', 'GPS座標Y']]
df = df.fillna(value='NULL')
df.tail()

,年,月,日,時,分,縣市,區,GPS座標X,GPS座標Y
10127,2017,11,30,22,45,臺中市,北區,24.1668,120.668
10128,2017,11,30,22,46,臺中市,豐原區,NULL,NULL
10129,2017,11,30,23,0,臺中市,豐原區,24.2472,120.73
10130,2017,11,30,23,28,臺中市,西區,24.1392,120.678
10131,2017,11,30,23,42,臺中市,南屯區,24.139,120.61


In [89]:
df.dtypes

年           int64
月           int64
日           int64
時           int64
分           int64
縣市         object
區          object
GPS座標X    float64
GPS座標Y    float64
dtype: object

In [91]:
## 建資料表語法
tcl = """create table tc_lite("""

for k in range(1):
    for i, j in zip(df.columns, df.dtypes):
        if j.name.find("int")==0:
            tcl = (tcl + """\n{} int NULL,""").format(i)
        elif j.name.find("float")==0:
            tcl = (tcl + """\n{} float NULL,""").format(i)
        else:
            tcl = (tcl + '\n{} varchar(20) NULL,').format(i)
            
tcl = tcl[:-1] + ');'   ##去掉最後的 逗點
print(tcl)

create table tc_lite(
年 int NULL,
月 int NULL,
日 int NULL,
時 int NULL,
分 int NULL,
縣市 varchar(20) NULL,
區 varchar(20) NULL,
GPS座標X float NULL,
GPS座標Y float NULL);


In [540]:
## 查詢現有資料表
#cursor.execute(str(tcl))
cursor.execute("show tables")
cursor.fetchall()

[('tc_lite', )]

In [105]:
### 建資料表 測試寫入全部
for i in df.index:
    cursor.execute("insert into tc_lite values({}, {}, {}, {}, {}, '{}', '{}', {}, {});" \
          .format(df.loc[i][0], df.loc[i][1], df.loc[i][2], df.loc[i][3], df.loc[i][4], df.loc[i][5], df.loc[i][6]\
                 , df.loc[i][7], df.loc[i][8]))
cursor.commit()

### 編碼有很大的問題，無法查詢指定資料

In [544]:
cursor.execute("select * from tc_lite limit 5;")
cursor.fetchall()

[(2017, 11, 1, 0, 7, '?箔葉撣?', '?批陸?\x80', 24.0738, 120.663),
 (2017, 11, 1, 0, 17, '?箔葉撣?', '瞏剖??\x80', 24.2013, 120.703),
 (2017, 11, 1, 1, 0, '?箔葉撣?', '瞏剖??\x80', 24.1946, 120.701),
 (2017, 11, 1, 1, 36, '?箔葉撣?', '?\ue919扈?\x80', 24.1824, 120.709),
 (2017, 11, 1, 1, 50, '?箔葉撣?', '?\ue919扈?\x80', 24.1736, 120.694)]

### ===========================================

## ===== 3版  成功 ======  最新版

In [447]:
# 資料庫連接
# CHARSET=UTF8; 和 cn.setencoding(encoding='utf-8') 都要設定

import pyodbc
cn = pyodbc.connect("DRIVER={MySQL ODBC 8.0 ANSI Driver}; SERVER=localhost; PORT=3306;DATABASE=taichung; UID=root; PASSWORD=123456;OPTION=3;CHARSET=UTF8;") 
cn.setencoding(encoding='utf-8')
cursor = cn.cursor()

In [449]:
cursor.execute("create database try")
cursor.execute("show databases")
cursor.fetchall()

[('information_schema', ),
 ('mysql', ),
 ('performance_schema', ),
 ('sakila', ),
 ('sys', ),
 ('taichung', ),
 ('try', ),
 ('world', )]

In [534]:
## 0513改版
q_tc = 'create table tc_lite('

for k in range(1):
    for i, j in zip(dff.columns, dff.dtypes):
        if type(dff.loc[0][i]) == str:
            q_tc = (q_tc + '\n{} varchar(20) NULL,').format(i)
        elif type(dff.loc[0][i]) == float:
            q_tc = (q_tc + '\n{} float NULL,').format(i)
        else:
            q_tc = (q_tc + '\n{} int NULL,').format(i)
            
q_tc = q_tc[:-1] + ');'   ##去掉最後的 逗點
print(q_tc)

create table tc_lite(
年 int NULL,
月 int NULL,
日 int NULL,
時 int NULL,
分 int NULL,
縣市 varchar(20) NULL,
區 varchar(20) NULL,
GPS座標X float NULL,
GPS座標Y float NULL);


In [535]:
#cursor.execute("use try;")
cursor.execute("drop table tc_lite")
cursor.execute(q_tc)
cursor.execute("show tables;")
cursor.fetchall()

[('tc_lite', )]

In [536]:
##### insert 3版 #########
for i in dff.index:
    cursor.execute("insert into tc_lite values({})".format(str(list(dff.loc[i]))[1:-1]))

cursor.commit()
cursor.execute("select * from tc_lite;")
cursor.fetchall()

[(2017, 11, 1, 0, 7, '?箔葉撣?', '?批陸?\x80', 24.0738, 120.663),
 (2017, 11, 1, 0, 17, '?箔葉撣?', '瞏剖??\x80', 24.2013, 120.703),
 (2017, 11, 1, 1, 0, '?箔葉撣?', '瞏剖??\x80', 24.1946, 120.701)]

In [539]:
cursor.execute("select GPS座標X from tc_lite;")
cursor.fetchall()

[(24.0738, ), (24.2013, ), (24.1946, )]